In [78]:
%load_ext line_profiler

In [79]:
from itertools import product
import numpy as np
import networkx as nx
import time

from adapt import adapt
from callbacks import MaxGradientStopper, ParameterStopper, EnergyErrorPrinter
from simulator import make_connectivity_pool, plus_state
from scipy.optimize import minimize

In [87]:
num_qubits = 6
ham = np.diag(np.random.uniform(size=2**num_qubits))

In [88]:
#pool = make_connectivity_pool(nx.path_graph(num_qubits))

In [89]:
def _two_local(i, j, a, b, n):
    p = [0] * n
    p[i] = a
    p[j] = b
    return p

pool = []
for i in range(num_qubits):
    for j in range(num_qubits):
        for a in range(4):
            for b in range(4):
                pool.append(_two_local(i, j, a, b, num_qubits))

In [90]:
def optimizer(*args, **kwargs):
    return minimize(*args, method='Nelder-Mead', **kwargs)

In [94]:
callbacks = [
    MaxGradientStopper(1e-5),
    ParameterStopper(10),
    EnergyErrorPrinter(np.min(np.linalg.eigvalsh(ham))),
    
]

In [95]:
def main():

    t_i = time.time()
    
    result = adapt(
        ham=ham,
        pool=pool,
        optimizer=optimizer,
        callbacks=callbacks,
        initial_state=plus_state(num_qubits),
        new_parameter=1e-2
    )

    t_f = time.time()

    print(f'Time elapsed: {np.floor(t_f-t_i)}s')

In [96]:
%lprun -f adapt main()


Energy error at step 2 reported: 0.3139355607613415
Log_10(|Energy error|) at step 2 reported: -0.5031594872063088

Energy error at step 3 reported: 0.24599827254491835
Log_10(|Energy error|) at step 3 reported: -0.609067942599238
Improvement from last (positive good): 0.06793728821642314
Log_10(|improvement|): -1.167891792410766

Energy error at step 4 reported: 0.1532492733100003
Log_10(|Energy error|) at step 4 reported: -0.8146015761779335
Improvement from last (positive good): 0.09274899923491806
Log_10(|improvement|): -1.0326907677409227

Energy error at step 5 reported: 0.14878335206299045
Log_10(|Energy error|) at step 5 reported: -0.8274456609384289
Improvement from last (positive good): 0.004465921247009841
Log_10(|improvement|): -2.3500889396368647

Energy error at step 6 reported: 0.11439181679651025
Log_10(|Energy error|) at step 6 reported: -0.9416050423894506
Improvement from last (positive good): 0.0343915352664802
Log_10(|improvement|): -1.4635484364816191

Energy err

In [ ]:
result.dataframe

In [ ]:
result.halt_reason

In [ ]:
result.step_history[-1]['energy'] - np.min(np.linalg.eigvalsh(ham))